# Capstone Project - Predicting Stock Prices Using Machine Learning 

Here, we will be gathering business/finance news articles from the New York Times dating from 1990 to 2019, and apply sentiment analysis to gauge the economic sentiment for each day:
<br>
<br>
- __Positive__ (Investors likely to __BUY__)
- __Neutral__ (Investors likely to take __NO ACTION__)
- __Negative__ (Investors likely to __SELL__)

## 1. Pre-Process the Data - Part 1. Gathering Data

### 1.1 Import All Required Libraries

In [3]:
# !pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 4.8 MB/s 


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
## Pandas and Numpy
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np

## API
import requests
import json

## Regex & Miscellaneous
import re as re
import time as timer
from datetime import datetime, date, time
from tqdm import tqdm
from collections import Counter
import pickle

## Sentiment Analysis
import spacy
import nltk
from spacy.lang.en.stop_words import STOP_WORDS
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [12]:
!cp drive/MyDrive/Load_MasterDictionary.py .
!cp drive/MyDrive/LoughranMcDonald_MasterDictionary_2021.csv .

In [13]:
import Load_MasterDictionary as MD ## LM Dictionary

file_path = ('LoughranMcDonald_MasterDictionary_2021.csv')

md, header, sent_cat, sent_dict, stopwords, total_doc = MD.load_masterdictionary(file_path, get_other = True)

### 1.2. Helper Functions for Extracting Data

In [14]:
## Helper Function for Creating A Dictionary
def create_dict(temp_dict, key, value):
    if key in temp_dict.keys():
        temp_dict[key].append(value)
    else: 
        temp_dict[key] = []
        temp_dict[key].append(value)
    return temp_dict

In [15]:
## Helper Function for Extracting Publication Dates and Abstract from the JSON Data.
def extract_data(r):
    r_dict = {}
    temp_json = r.json()
    doc = temp_json["response"]["docs"]
    length = list(range(len(temp_json["response"]["docs"])))
    
    for i in tqdm (length, 
                   desc="Loading…"):

        news_desk = doc[i]['news_desk']
        lookup = r"[bB]usiness|[fF]inan|[nN]one|[mM]oney"
        match = re.search(lookup, news_desk)
        
        if match:
            date = doc[i]["pub_date"]
            date = date[:10]
            abstract = doc[i]["abstract"]
            lead_paragraph = doc[i]['lead_paragraph']
            
            if len(abstract) < 700:
                value = abstract
            else:
                value = lead_paragraph
            
            r_dict = create_dict(r_dict, date, value)
            
        else:
            continue
    
    return r_dict

In [16]:
### Helper Function for Filtering the Data Based on Keywords
keywords = ["Standard & Poor", "S&P 500", "stock market", "index", ".INX", "Wall Street"]

def filter_keywords(r_dict, keywords):
    f_dict = {}
    
    for date, abstract in r_dict.items():
        
        for i in range(len(abstract)):
            
            for keyword in keywords:
                
                if (keyword in abstract[i]) and ("COMPANY REPORT" not in abstract[i]):                    
                    
                    f_dict = create_dict(f_dict, date, abstract[i])
                    
                else:
                    continue
    
    return f_dict

In [17]:
### Helper Function for Gathering Data from API
def get_data(start_year, end_year):
    
    data_dict = {}
    
    years = range(start_year, end_year+1)
    months = range(1, 13)
    
    for year in tqdm(years, desc="Loading…"):
        
        for month in months:
            date = f"{year}-{month}"
            print(f"Gathering Data for Year: {year}, Month: {month}")
                
            url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?api-key={key}"
            r = requests.get(url)
            r_dict = extract_data(r)
            f_dict = filter_keywords(r_dict, keywords)
            
            data_dict = create_dict(data_dict, date, f_dict)
        
            timer.sleep(5) # Sleep for 5 seconds
    
    return data_dict

### 1.3. API Calls & Gather Data From New York Times

In [18]:
key = "3aD0nGomAVgo2sv2dJiKPNG0iEHoR4AL "

In [19]:
dict_9099 = get_data(1990, 1999) ## Gather Data from 1990 to 1999

Loading…:   0%|          | 0/10 [00:00<?, ?it/s]

Gathering Data for Year: 1990, Month: 1



Loading…: 100%|██████████| 8879/8879 [00:00<00:00, 493411.57it/s]


Gathering Data for Year: 1990, Month: 2



Loading…: 100%|██████████| 8468/8468 [00:00<00:00, 279356.35it/s]


...


Loading…: 100%|██████████| 8511/8511 [00:00<00:00, 377414.19it/s]


Gathering Data for Year: 1999, Month: 11



Loading…: 100%|██████████| 7874/7874 [00:00<00:00, 391739.02it/s]


Gathering Data for Year: 1999, Month: 12



Loading…: 100%|██████████| 10/10 [12:25<00:00, 74.56s/it]


In [20]:
dict_0009 = get_data(2000, 2009) ## Gather Data from 1990 to 1999

Loading…:   0%|          | 0/10 [00:00<?, ?it/s]

Gathering Data for Year: 2000, Month: 1



Loading…: 100%|██████████| 8502/8502 [00:00<00:00, 379067.03it/s]


Gathering Data for Year: 2000, Month: 2



Loading…: 100%|██████████| 7953/7953 [00:00<00:00, 363226.84it/s]


...


Loading…: 100%|██████████| 15463/15463 [00:00<00:00, 396237.33it/s]


Gathering Data for Year: 2009, Month: 11



Loading…: 100%|██████████| 9432/9432 [00:00<00:00, 429596.42it/s]


Gathering Data for Year: 2009, Month: 12



Loading…: 100%|██████████| 10/10 [12:55<00:00, 77.55s/it]


In [21]:
dict_1019 = get_data(2010, 2019) ## Gather Data from 1990 to 1999

Loading…:   0%|          | 0/10 [00:00<?, ?it/s]

Gathering Data for Year: 2010, Month: 1



Loading…: 100%|██████████| 9820/9820 [00:00<00:00, 425628.45it/s]


Gathering Data for Year: 2010, Month: 2



Loading…: 100%|██████████| 9298/9298 [00:00<00:00, 459272.19it/s]


...


Loading…: 100%|██████████| 4989/4989 [00:00<00:00, 418976.91it/s]


Gathering Data for Year: 2019, Month: 11



Loading…: 100%|██████████| 4420/4420 [00:00<00:00, 302225.65it/s]


Gathering Data for Year: 2019, Month: 12



Loading…: 100%|██████████| 10/10 [12:49<00:00, 76.95s/it]


## 2. Pre-Process the Data - Part 2. NLP: Tokenisation & Lemmatisation

### 2.1 Load spaCy and the English Model

In [23]:
# !python -m spacy download en_core_web_md

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 417 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


In [24]:
# load spaCy and the English model
nlp = spacy.load("en_core_web_md")

### 2.2 Define Helper Functions for Filtering Stop Words

In [25]:
def create_doc(text):
   ## Clean up the text
    text = text.lower()
    text = re.sub(r"[ \t]+", " ", text)
    text = text.lstrip()
    text = re.sub('[A-Z]+', lambda x: x.group(0).title(), text)
    text = re.sub(r'\([^)]*\)', '', text)
    text = re.sub(r"[']|[,]|[.]|[:]|[;]", "", text)
    text = re.sub(r'(\s\s+|\n\n+)', r'\1', text)
#     text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text)
    
    doc = nlp(text) ## Tokenisation
    
    return doc

In [26]:
def filter_text(doc):
    filtered_doc =[] 
    token_list = []
    lemma_list = []
    
    for token in doc:
        if token.is_stop == False or token.dep_ in ["det", "neg", "npadvmod:"] and token.pos_ != "DET":
            ## Remove Stop Words
            token_list.append(token)
    
    for word in token_list:
        lemma_list.append(word.lemma_) ## Lemma version of the text
    
    for word in lemma_list: ## Remove Punctuations and Non-alphabetical texts
        lexeme = nlp.vocab[word]
        if word not in " " and lexeme.is_punct == False and lexeme.is_alpha == True:
            filtered_doc.append(word)
                    
    return filtered_doc ## Return original tokenised text and filtered text

In [27]:
## Helper Function for Creating a Dataframe
def create_df(data_dict):
    ## Turn the dictionary into a dataframe
    text_dict = {}
    count = 0
    
    for key, value in data_dict.items():
        for date, texts in value[0].items():
            for index in range(len(texts)):
                txt = texts[index]
                doc = create_doc(txt)
                f_doc = filter_text(doc)
#                 f_doc = " ".join(f_doc)
                
                temp_list = [date, txt, f_doc]
                text_dict = create_dict(text_dict, count, temp_list)
                
                count += 1   
        
    df = pd.DataFrame.from_dict(text_dict,orient='index')
    df[['date',"original", 'texts']] = pd.DataFrame(df[0].tolist(), index= df.index)
    df = df.drop([0], axis=1)
    df = df[~df.original.duplicated(keep='first')]
    df = df.reset_index(drop=True)
    
    ## Drop articles that are longer than 1000 characters for simplicity
    long = []
    for index in range(len(df)):
        if len(df.original[index]) > 1000:
            long.append(index)
            
    df = df.drop(labels = long, axis = 0)
    df = df.reset_index(drop=True)
    
    for i in df.index:
        if df.texts[i][0] == "lead":
            df.texts[i].pop(0)
        else:
            continue

    df["shorts"] = ""
    for i in df.index:
        df.shorts[i] = " ".join([word for word in df.texts[i]])
    
    return df

In [28]:
dict_list = [dict_9099, dict_0009, dict_1019]
df_list = []

for d in dict_list:
    df = create_df(d)
    df_list.append(df)

In [31]:
df_list[2]

,date,original,texts,shorts
0,2010-01-01,With the possibility of hefty profits and bonu...,"[possibility, hefty, profit, bonus, bank, rare...",possibility hefty profit bonus bank rarely div...
1,2010-01-04,Veteran value investor Martin J. Whitman will ...,"[veteran, value, investor, martin, j, whitman,...",veteran value investor martin j whitman relinq...
2,2010-01-05,China’s securities regulator may introduce fut...,"[china, security, regulator, introduce, future...",china security regulator introduce future cont...
3,2010-01-05,Wall Street ushered in 2010 on Monday with its...,"[wall, street, usher, monday, big, stock, mark...",wall street usher monday big stock market rall...
4,2010-01-05,Is the return of the Wall Street bonus helping...,"[return, wall, street, bonus, helping, spur, c...",return wall street bonus helping spur comeback...
...,...,...,...,...
2188,2019-11-01,The S&P 500 set a new high for the third time ...,"[set, new, high, time, week, nasdaq, composite...",set new high time week nasdaq composite rise r...
2189,2019-12-06,Saudi Arabia’s state-owned oil company priced ...,"[saudi, arabia, state, own, oil, company, pric...",saudi arabia state own oil company price offer...
2190,2019-12-20,The Wall Street bank would pay a fine of as mu...,"[wall, street, bank, pay, fine, billion, subsi...",wall street bank pay fine billion subsidiary p...
2191,2019-12-23,Wall Street strategists are issuing prediction...,"[wall, street, strategist, issue, prediction, ...",wall street strategist issue prediction ignore...


In [32]:
# df.to_pickle("df_9099.pkl") 

## 3.0 Pre-Processing - Part 3. Labeling Positive, Neutral and Negative News

In [33]:
## Convert the LM Dictionary into positive/negative word lists
positive = []
negative = []

for sent, value in sent_dict.items():
    if sent == "negative" or sent == "uncertainty":
        for word in value.keys():        
            negative.append(word.lower())
    
    elif sent == "positive" :
        for word in value.keys():        
            positive.append(word.lower())

In [34]:
!cp drive/MyDrive/positive.xlsx .
!cp drive/MyDrive/negative.xlsx .

In [35]:
GI_post_csv = "positive.xlsx" ## GI Dictionary
GI_neg_csv = "negative.xlsx"

In [36]:
GI_post = pd.read_excel(GI_post_csv, header = None)
GI_neg = pd.read_excel(GI_neg_csv, header = None)

GI_post_list = list(GI_post[0])
GI_neg_list = list(GI_neg[0])

In [37]:
## Extra words defined by the user to be relevant in the analysis

positive_extra = [r"[bB]ull", "positive", "rise", "stock", "high", "up", "climb", "buy", "surge", "recover",
                  "soar", "up", "blue", "chip"]

negative_extra = [r'[bB]ear', "negative", "drop", "fall", "low", 
                  "burst", "tight", "bubble", "crash", "down", "slide", "sell", "plunge", "dive",
                  "stagflation", "recession", "slow", "inflation", "down"]

In [38]:
## Combine all lists into a single list
positive += (positive_extra + GI_post_list)
negative += (negative_extra + GI_neg_list)

## Remove duplicates and convert to a final list
positive_words = list(set(positive))
negative_words = list(set(negative))

In [39]:
## Helper Function for Sentiment Labelling Using Custom Made Lexicon Dictionary
def get_lexicon_score(df):
    labels = {}
    for i in df.index:
    
        for word in df.texts[i]:            
            if word in positive_words:
                labels = create_dict(labels, i, "+")
        
            elif word in negative_words:
                labels = create_dict(labels, i, "-") 
        
            else:
                labels = create_dict(labels, i, "0")
    
    scores = {}
    for key, value in labels.items():
        plus = value.count("+")
        minus = value.count("-")
        neutral = value.count("0")
        total = len(value)
        pos_score = plus/total
        neg_score = minus/total
        neut_score = neutral/total
        
        r = np.array([pos_score, neg_score])
    
        score = max([pos_score, neg_score])
        if neg_score > pos_score:
            score = -score
            
        scores[key] = score 
    
    return scores

In [40]:
## Helper Function for Getting Polarity Score using TextBlob
def get_polar_score(df):
    polar = []
    for text in df.shorts:
        textBlob = TextBlob(text)
        polar.append(textBlob.polarity * textBlob.subjectivity)
    
    return polar

In [41]:
## Function to print sentiments of the sentence.
def vader_score(text):
    
    # Create a SentimentIntensityAnalyzer object.
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(text)
    comp_score = sentiment_dict['compound']

    return comp_score

In [42]:
## Helper Function for Getting Vader Scores for the Entire Dataframe
def get_vader_score(df):    
    vader= []
    for text in df.shorts:
        vader.append(vader_score(text))
    
    return vader

In [43]:
## Helper Function for Getting the Final Aggregated Score
def get_agg_scores(df):
    scores = get_lexicon_score(df)
    polar = get_polar_score(df)
    vader = get_vader_score(df)
    
    agg_scores = []
    weight = [0.1, 0.05, 0.85]
    
    for i in range(len(polar)):
        value = np.sum([weight[0]*polar[i], weight[1]*vader[i], weight[2]*scores[i]])
        agg_scores.append(value)
    
    return agg_scores

In [44]:
new_df_list = []

for df in df_list:
    agg_scores = get_agg_scores(df)
    df["sentiment_score"] = agg_scores
    
    df_temp = df.copy()
    df_group = df_temp.groupby(["date"]).agg({'sentiment_score': ["sum"]})
    new_df_list.append(df_group)

In [47]:
new_df_list[2]

,sentiment_score
,sum
date,
2010-01-01,0.258499
2010-01-04,0.064222
2010-01-05,0.721483
2010-01-06,0.235975
2010-01-07,0.186924
...,...
2019-11-01,0.196258
2019-12-06,0.135332


In [48]:
df_list[0].to_pickle("df_9099.pkl") 
df_list[1].to_pickle("df_0009.pkl") 
df_list[2].to_pickle("df_1019.pkl") 

In [49]:
new_df_list[0].to_pickle("df_9099_score.pkl") 
new_df_list[1].to_pickle("df_0009_score.pkl") 
new_df_list[2].to_pickle("df_1019_score.pkl") 

In [ ]:
from google.colab import drive
drive.flush_and_unmount()